In [2]:
!pip install stop_words

Processing c:\users\essor\appdata\local\pip\cache\wheels\eb\03\0d\3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062\stop_words-2018.7.23-py3-none-any.whl


In [18]:
!pip install -U spacy

Requirement already up-to-date: spacy in c:\programs\anaconda3\lib\site-packages (2.3.4)


In [9]:
import pandas as pd
import numpy as np

import re

import string
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

import spacy
import nltk
from nltk.tokenize import word_tokenize

from gensim.models import Word2Vec

import warnings 
warnings.filterwarnings("ignore")
import os

#### Задание 1.
Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)
С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?
Повторим шаги из заданий 1 и 2, используя библиотеку nltk.


In [10]:
#Предобработанные данные взяты с прошлого ДЗ
combine_df = pd.read_pickle('preproc.pkl')
combine_df.head(3)

,id,label,tweet,clean_tweet,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"


In [16]:
import en_core_web_sm

In [20]:
nlp = en_core_web_sm.load()

In [22]:
%%time
combine_df['NER'] = combine_df['clean_tweet'].apply(lambda x:  nlp(x))

ner = combine_df['NER'].tolist()

NER = []
for doc in ner:
    for ent in doc.ents:
        NER.append((ent.text, ent.label_))

ner_df = pd.DataFrame(NER, columns=['word', 'ner'])
ner_df.head()

Wall time: 4min 14s


,word,ner
0,tomorrow,DATE
1,the next school year,DATE
2,the year,DATE
3,allin cavs champions cleveland,FAC
4,ireland,GPE


In [23]:
#TOP20 NER
ner_top_20 = ner_df.ner.value_counts().head(20)
ner_top_20

DATE           11951
PERSON          7280
ORG             5280
GPE             4825
TIME            2153
NORP            1598
CARDINAL        1235
ORDINAL          664
FAC              218
LOC              186
EVENT            124
PRODUCT           81
LANGUAGE          38
QUANTITY          21
LAW               15
MONEY              9
WORK_OF_ART        4
PERCENT            4
Name: ner, dtype: int64

In [24]:
#TOP20 PERSON
pers_top_20 = ner_df.loc[ner_df['ner'] == 'PERSON']

pers_top_20 = pers_top_20.word.value_counts().head(20)
pers_top_20

polar bear                            97
hu                                    59
yo                                    55
blur sun                              54
hillary                               45
tgif                                  29
sikh temple                           28
donald trump                          25
magnettherapy                         19
bihday                                18
carl paladino                         18
christina grimmie                     18
sirf ashiq                            17
clinton                               17
dwd wetterwarnung starkes gewitter    17
warnung wetter                        17
ali                                   17
ripchristina                          17
karen iqbal galib                     17
vegan                                 16
Name: word, dtype: int64

In [25]:
#TOP20 ORGANIZATION
org_top_20 = ner_df.loc[ner_df['ner'] == 'ORG']

org_top_20 = org_top_20.word.value_counts().head(20)
org_top_20

sta                        215
aww                        115
gop                         77
chase                       61
na                          50
nba                         42
house                       39
islam                       36
cnn                         33
update social analytics     32
euro                        31
udtapunjab                  30
eu                          29
tgif                        25
hea                         24
fed                         23
fbi                         21
congress                    21
amazon                      20
disney                      20
Name: word, dtype: int64

#### Задание 2.
Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор

#### Задание 3.
Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.
